In [5]:
import tiktoken
import torch

for rapid prototyping with new ideas 

In [2]:
import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

# Save to a file
with open('input.txt', 'w', encoding='utf-8') as f:
    f.write(response.text)

print("Downloaded successfully!")

Downloaded successfully!


In [3]:
with open("input.txt", 'r') as f:
    text = f.read()

data = text[:1000]
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [17]:
enc = tiktoken.get_encoding("gpt2")

tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


`198` is `\n` or newline just to verify correctness sometimes. 

In [18]:
# create inputs and outputs using .view operations
buf = torch.tensor(tokens[:24+1])

x = buf[:-1].view(4,6)
# output is input shifted by 1
y = buf[1:].view(4,6) 

print(f"input batch = {x} \noutput batch = {y}")

input batch = tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]]) 
output batch = tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])
